# Exercícios de Redução de Dimensionalidade

## Introdução

Utilizaremos os dados de um [distribuidor por atacado Português](https://archive.ics.uci.edu/ml/datasets/Wholesale+customers) para agrupamento. Essa base de dados tem o nome de `Wholesale_Customers_Data`.

Essa base contêm os seguintes atributos:

* Fresh: annual spending (m.u.) on fresh products
* Milk: annual spending (m.u.) on milk products
* Grocery: annual spending (m.u.) on grocery products
* Frozen: annual spending (m.u.) on frozen products
* Detergents_Paper: annual spending (m.u.) on detergents and paper products
* Delicatessen: annual spending (m.u.) on delicatessen products
* Channel: customer channel (1: hotel/restaurant/cafe or 2: retail)
* Region: customer region (1: Lisbon, 2: Porto, 3: Other)

Nessa base, os valores de gastos são dados em uma unidade arbitrária (m.u. = monetary unit, unidade monetária).

## Exercício 1

* Importe os dados e verifique os seus tipos.
* Remova as colunas channel e region.
* Converta o restante para floats se necessário.
* Copie a variável original de data (usando o método `copy`) para preservá-la.

In [1]:
import pandas as pd
import numpy as np

filepath = 'data/Wholesale_Customers_Data.csv'
data = pd.read_csv(filepath, sep=',')

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data = data.drop(???, axis=1)

In [ ]:
# tipos das variáveis
data.???

In [ ]:
# Converte para floats
for col in data.columns:
    data[col] = data[col].astype(np.float)

Faça uma cópia do original.

In [ ]:
data_orig = data.???

## Exercício 2

Vamos repetir o procedimento de transformação e escala da atividade anterior (Exercício 2):

* Examine a correlação e viés.
* Faça a transformação e escalonamento necessários.
* Visualize as correlações par a par.

In [ ]:
# matriz de correlação
corr_mat = data.???

# Sete a diagonal com 0.0
for x in range(corr_mat.shape[0]):
    corr_mat.iloc[x,x] = ???
    
corr_mat

Retorne o id de máxima correlação:

In [ ]:
corr_mat.???.???

Examine os atributos com viés e aplique log1p:

In [ ]:
log_columns = data.???.sort_values(ascending=False)
log_columns = log_columns.loc[log_columns > 0.75]

log_columns

In [ ]:
# The log transformations
for col in log_columns.index:
    data[col] = ???(data[col])

Vamos usar o `MinMaxScaler` para fazer o escalonamento.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()

for col in data.columns:
    data[col] = mms.fit_transform(???).squeeze() # squeeze transforma em vetor

Vamos plotar as relações.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
sns.set_context('notebook')
sns.set_palette('dark')
sns.set_style('white')

sns.pairplot(data);

## Exercício 3

* Usando a função [pipeline](http://scikit-learn.org/stable/modules/pipeline.html) do Scikit-Learn, recrie o procedimento acima de pré-processamento (transformação e escala) usando pipeline. Para utilizar uma função de transformação externa (log1p do Numpy) você precisará da classe chamada [`FunctionTransformer`](http://scikit-learn.org/stable/modules/preprocessing.html#custom-transformers).
* Use o pipeline para transformar a cópia dos dados originais.
* Compare com o resultado do exercício anterior para verificar se funcionou.

In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

# Crie a transformação log do Numpy para ser utilizada no pipeline
log_transformer = FunctionTransformer(???)

# O pipeline
estimators = [('log1p', log_transformer), ('minmaxscale', MinMaxScaler())]
pipeline = Pipeline(estimators)

# Converte a cópia dos dados originais
data_pipe = pipeline.fit_transform(???)

Os resultados são idênticos. Note que podemos acrescentar qualquer modelo dentro do pipeline, como algoritmos de classificação e regressão.

In [ ]:
np.allclose(data_pipe, data)

## Exercício 4

* Aplique o PCA com `n_components` variando entre 1 e 5. 
* Armazene a quantidade de variância explicada (explained_variance_ratio_.sum()) para cada número de dimensões.
* Vamos estimar a importância dos atributos gerados com np.abs(components_).sum(axis=0)/ soma de todos.
* Plote os valores acima.

In [ ]:
from sklearn.decomposition import PCA

pca_list = list()
feature_weight_list = list()

# Fit a range of PCA models

for n in range(???, ???):
    
    # Create and fit the model
    PCAmod = PCA(n_components=???)
    PCAmod.fit(???)
    
    # Store the model and variance
    pca_list.append(pd.Series({'n':n, 'model':PCAmod,
                               'var': PCAmod.explained_variance_ratio_.sum()}))
    
    # Calculate and store feature importances
    abs_feature_values = np.abs(PCAmod.components_).sum(axis=0)
    feature_weight_list.append(pd.DataFrame({'n':n, 
                                             'features': data.columns,
                                             'values':abs_feature_values/abs_feature_values.sum()}))
    
pca_df = pd.concat(pca_list, axis=1).T.set_index('n')
pca_df

Vamos criar uma tabela para verificar os resultados:

In [ ]:
features_df = (pd.concat(feature_weight_list)
               .pivot(index='n', columns='features', values='values'))

features_df

E vamos plotar os resultados:

In [ ]:
sns.set_context('talk')

ax = pca_df['var'].plot(kind='bar')

ax.set(xlabel='Number of dimensions',
       ylabel='Percent explained variance',
       title='Explained Variance vs Dimensions');

E da importância dos atributos:

In [ ]:
ax = features_df.plot(kind='bar')

ax.set(xlabel='Number of dimensions',
       ylabel='Relative importance',
       title='Feature importance vs Dimensions');

## Exercício 5

Vamos verificar como a acurácia de nosso modelo pode mudar se incluírmos um `PCA` no nosso pipeline de processamento. Vamos criar um pipeline com os seguintes passos:
<ol>
  <li>Escalonar</li>
  <li>`PCA(n_components=n)`</li>
  <li>`LogisticRegression`</li>
</ol>

* Para isso utilizaremos a base de dados Human Activity.
* Escreva uma função que pega um valor  `n`  e constrói o pipeline acima, faça a predição da coluna  "Activity" em um StratifiedShuffleSplit de 5 pastas (fold), retorne a acurácia média da base de teste.
* Chame a função acima para vários valores de n.
* Plote a acurácia média pela dimensão.

In [2]:
filepath = 'data/Human_Activity_Recognition_Using_Smartphones_Data.csv'
data2 = pd.read_csv(filepath, sep=',')

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


X = data2.drop('Activity', axis=1)
y = data2.Activity
sss = StratifiedShuffleSplit(n_splits=???, random_state=42)

def get_avg_score(n):
    pipe = [
        ('scaler', ???),
        ('pca', ???),
        ('estimator', ???)
    ]
    pipe = Pipeline(pipe)
    scores = []
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        pipe.fit(???, ???)
        scores.append(accuracy_score(???, pipe.predict(???)))
    return np.mean(scores)


ns = [10, 20, 50, 100, 150, 200, 300, 400]
score_list = [get_avg_score(n) for n in ns]

In [ ]:
sns.set_context('talk')

ax = plt.axes()
ax.plot(ns, score_list)
ax.set(xlabel='Number of Dimensions',
       ylabel='Average Accuracy',
       title='LogisticRegression Accuracy vs Number of dimensions on the Human Activity Dataset')
ax.grid(True)